In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, DatasetDict
import pandas as pd

/opt/miniconda3/envs/uni311/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
model_name = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map=device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [4]:
df = pd.read_csv("dataset_cleaned.csv")
df = pd.read_csv("gold_labeled_dataset_1.csv", sep=";")
df = df.rename(columns={"Sentence": "source", "Traductions": "target"})

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=41)


# row = df.iloc[12] # from 10 ahead to avoid using sentences given as examples 

# author = row["Author"]
# date = row["Date"]
# region = row["Region"]
# sentence = row["Sentence"]


NON IN-CONTEXT LEARNING

In [6]:
system_prompt = "Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo."

In [ ]:
# output = pipe(messages, max_new_tokens=300)
# assistant_reply = output[0]["generated_text"][-1]["content"]



# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": user_prompt}
# ]

# print("Frase antica:")
# print(sentence)
# print("Traduzione moderna:")
# print(assistant_reply)


i = 1
for sample in dataset["test"]:


    input_sentence = sample["source"]
    target_sentence = sample["target"]

    author = sample["Author"]
    date = sample["Date"]
    region = sample["Region"]


    print(f"Sentence {i}")
    print(f"\tItaliano Arcaico -> {input_sentence}")
    
    user_prompt = f"Modernizza il seguente testo antico in italiano contemporaneo:\n\n\"{input_sentence}\"\n\n scrivi solo ed esclusivamente la traduzione"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    output = pipe(messages, max_new_tokens=300)
    translation = output[0]["generated_text"][-1]["content"]

    print(f"\tItaliano moderno -> {translation}")
    print(f"\tGOLD LABEL       -> {target_sentence}")

    print(f"-----------------------------------------")
    i+=1

Sentence 1
	Italiano Arcaico -> l'armi et insieme con loro passaseno tra li nimici, perçò se alcuno non avesse ardire de questo et sì avevano questo animo.
	Italiano moderno -> L'esercito, insieme a loro, passava tra i nemici, e si chiedevano se nessuno avesse un'idea di questo gruppo, perché avevano questo animo.
	GOLD LABEL       -> Le armi, insieme a loro, passarono tra i nemici; infatti, se qualcuno non avesse avuto coraggio, loro ce l’avevano.
-----------------------------------------
Sentence 2
	Italiano Arcaico -> Gli uomini spessamente a stare fermi nella bugia incontra la verità.
	Italiano moderno -> “Essere fermi nella bugia porta alla scoperta della verità.”

	GOLD LABEL       -> Gli uomini spesso, per restare fermi nella menzogna, si oppongono alla verità.
-----------------------------------------
Sentence 3
	Italiano Arcaico -> e l' acconciamento a fare grandissime cose, cioè a ttenere pace et amare Idio e 'l proximo, a ffare cittadi, castella e magioni


IN-CONTEXT LEARNING

In [ ]:
system_prompt = """ Sei un esperto di lingua italiana. Il tuo compito è modernizzare testi antichi in italiano contemporaneo. Di seguito alcuni esempi:

Testo antico: "quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno"
Traduzione moderna: "Quella guerra fu ben condotta per raggiungere il suo scopo. Dall'altra parte, Aiace era un cavaliere leale e valoroso nelle armi, di grande statura, ma non molto saggio."

Testo antico: "crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi."
Traduzione moderna: "È crudele e si vendica di ogni colpa, come stabilisce la legge, e non perdona alcun cavaliere che sbagli."

Testo antico: "Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere."
Traduzione moderna: "Ponzio Aufidiano, cavaliere romano, non era dotato di maggiore coraggio."

Testo antico: "Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati."
Traduzione moderna: "Se questo è il volere di tutti, e se i tempi richiedono Pompeo come guida e non come alleato, allora non tratterrò più il destino."

Testo antico: "Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire."
Traduzione moderna: "Il compito di quest'arte sembra essere quello di parlare con astuzia per far credere; lo scopo è dunque persuadere attraverso il linguaggio."

Testo antico: "Ecco e larghi ventipiovoli caggiono delle risolute nebbie; e potresti credere che tutto il cielo cadesse nel mare"
Traduzione moderna: "Ecco che ampie piogge ventose cadono dalle nebbie diradate; potresti pensare che tutto il cielo stia precipitando nel mare."

Testo antico: "Però che or chi spererebbe quello che eziandio questi che non vogliono ancora credere in Cristo, già veggiono con noi, e perché nol possono negare, stridono colli denti."
Traduzione moderna: "Chi ormai oserebbe sperare ciò che persino quelli che ancora non vogliono credere in Cristo vedono con i propri occhi insieme a noi, e poiché non possono negarlo, digrignano i denti."

Testo antico: "I vendimenti de' morti et le presure de' vivi fece la frode d'uno feroce re."
Traduzione moderna: "Fu l'inganno di un re feroce a provocare le vendette dei morti e le catture dei vivi."

Testo antico: "Acciocché quegli, il quale ora per le sue gran reità è feroce e onorevole, egli d'ogni male afflitto e tormentato della impietà verso il mio padre."
Traduzione moderna: "Affinché colui che ora, per i suoi grandi misfatti, è temuto e rispettato, sia tormentato da ogni male a causa della sua empietà verso mio padre."

Testo antico: "Gli uomini spessamente a stare fermi nella bugia incontra la verità."
Traduzione moderna: "Gli uomini spesso restano fermi nella menzogna, opponendosi alla verità."
"""

In [ ]:
i = 1
for sample in dataset["test"]:


    input_sentence = sample["source"]
    target_sentence = sample["target"]

    author = sample["Author"]
    date = sample["Date"]
    region = sample["Region"]


    print(f"Sentence {i}")
    print(f"\tItaliano Arcaico -> {input_sentence}")
    
    user_prompt = f"""Modernizza il seguente testo antico in italiano contemporaneo.
    Contesto: Autore: {author}, Data: {date}, Regione: {region}
    Testo antico: "{input_sentence}"

    Scrivi solo ed esclusivamente la traduzione moderna:"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    output = pipe(messages, max_new_tokens=300)
    translation = output[0]["generated_text"][-1]["content"]

    print(f"\tItaliano moderno -> {translation}")
    print(f"\tGOLD LABEL       -> {target_sentence}")

    print(f"-----------------------------------------")
    i+=1

Frase antica:
Corbio nipote d' Ortensio menò sua vita più bassa e più viziosa
Traduzione moderna:
“Corbio, nipote di Ortensio, morì più velocemente e con una vita più sgradevole.”
